In [37]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime
import numpy as np

stocks = ['AMD', 'NVDA', 'META', 'GOOGL', 'TSLA', 'GM', 'AAPL', 'MSFT', 'NFLX', 'DIS', 'JPM', 'BAC', 'KO', 'PEP']
start = datetime(2016, 1, 1)
end = datetime(2018, 12, 30)

stock_prices         = yf.download(stocks, start , end, auto_adjust = False)
stock_prices = stock_prices.resample('W').last()
stock_prices.index   = stock_prices.index.tz_localize(None)      # change yf date format to match pdr
stock_prices         = stock_prices.filter(like='Adj Close')     # reduce to just columns with this in the name
stock_prices.columns = stock_prices.columns.get_level_values(1)  # tickers as col names, works no matter order of tics

stock_prices = stock_prices.unstack(level=0).reset_index()
stock_prices.columns = ['Ticker', 'Date', 'Adj Close']
stock_prices['Return'] = stock_prices.groupby('Ticker')['Adj Close'].pct_change()
stock_prices.set_index('Date', inplace=True) 
print(stock_prices.head(10))
stock_prices.to_csv("outputs/returns_weekly_large.csv", index=True)

[*********************100%***********************]  14 of 14 completed

           Ticker  Adj Close    Return
Date                                  
2016-01-10   AAPL  21.936222       NaN
2016-01-17   AAPL  21.974682  0.001753
2016-01-24   AAPL  22.945250  0.044168
2016-01-31   AAPL  22.022194 -0.040229
2016-02-07   AAPL  21.386499 -0.028866
2016-02-14   AAPL  21.379673 -0.000319
2016-02-21   AAPL  21.845985  0.021811
2016-02-28   AAPL  22.043875  0.009058
2016-03-06   AAPL  23.431437  0.062945
2016-03-13   AAPL  23.260832 -0.007281


In [38]:
pivot_data = stock_prices.pivot(columns='Ticker', values='Adj Close')
print(pivot_data)

Ticker           AAPL        AMD        BAC         DIS         GM      GOOGL  \
Date                                                                            
2016-01-10  21.936222   2.140000  12.412969   92.494095  23.942945  36.371613   
2016-01-17  21.974682   2.030000  11.808654   87.508263  23.975378  35.355469   
2016-01-24  22.945250   2.020000  11.073674   90.304070  23.740248  37.095650   
2016-01-31  22.022194   2.200000  11.547329   89.297577  24.032133  37.886368   
2016-02-07  21.386499   1.980000  10.575524   87.508263  23.140255  35.020569   
...               ...        ...        ...         ...        ...        ...   
2018-12-02  42.595020  21.299999  24.232344  111.835831  34.620541  55.218502   
2018-12-09  40.188354  19.459999  21.819447  109.278023  31.979309  52.080006   
2018-12-16  39.470390  19.900000  21.004333  109.492691  32.357269  52.335281   
2018-12-23  35.952213  16.930000  20.051929  101.705261  30.402925  49.326672   
2018-12-30  37.264076  17.82

In [39]:
pairs =[
    ('AAPL', 'MSFT'),
    ('GOOGL', 'META'),
    ('AMD', 'NVDA'),
    ('TSLA', 'GM'),
    ('KO', 'PEP'),
    ('NFLX', 'DIS'),
    ('JPM', 'BAC'),
    ('GOOGL', 'AAPL'),
    ('META', 'NFLX'),
    ('TSLA', 'NVDA')
]

spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = np.log(pivot_data[ticker_1]) - np.log(pivot_data[ticker_2])

    spread_mean = spread.mean()
    spread_std = spread.std()
    z_spread = (spread - spread_mean) / spread_std
    
    pair_df = z_spread.reset_index()
    pair_df['Date'] = pd.to_datetime(pair_df['Date'])  # Convert Date column to datetime
    pair_df.set_index('Date', inplace=True)
    pair_df['Ticker Pair'] = f'{ticker_1}-{ticker_2}'
    pair_df.columns = ['Spread', 'Ticker Pair']
    spread_list.append(pair_df[['Ticker Pair', 'Spread']])

spread_df = pd.concat(spread_list)

In [40]:
pivot_data = stock_prices.pivot(columns='Ticker', values='Return')
print(pivot_data)

Ticker          AAPL       AMD       BAC       DIS        GM     GOOGL  \
Date                                                                     
2016-01-10       NaN       NaN       NaN       NaN       NaN       NaN   
2016-01-17  0.001753 -0.051402 -0.048684 -0.053904  0.001355 -0.027938   
2016-01-24  0.044168 -0.004926 -0.062241  0.031949 -0.009807  0.049220   
2016-01-31 -0.040229  0.089109  0.042773 -0.011146  0.012295  0.021316   
2016-02-07 -0.028866 -0.100000 -0.084158 -0.020038 -0.037112 -0.075642   
...              ...       ...       ...       ...       ...       ...   
2018-12-02  0.036508  0.099071  0.053022  0.030425  0.056220  0.077225   
2018-12-09 -0.056501 -0.086385 -0.099573 -0.022871 -0.076291 -0.056838   
2018-12-16 -0.017865  0.022611 -0.037357  0.001964  0.011819  0.004902   
2018-12-23 -0.089135 -0.149246 -0.045343 -0.071123 -0.060399 -0.057487   
2018-12-30  0.036489  0.052569  0.043646  0.029553  0.028502  0.055919   

Ticker           JPM        KO      M

In [41]:
pairs =[
    ('AAPL', 'MSFT'),
    ('GOOGL', 'META'),
    ('AMD', 'NVDA'),
    ('TSLA', 'GM'),
    ('KO', 'PEP'),
    ('NFLX', 'DIS'),
    ('JPM', 'BAC'),
    ('GOOGL', 'AAPL'),
    ('META', 'NFLX'),
    ('TSLA', 'NVDA')
]


spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = pivot_data[ticker_1] - pivot_data[ticker_2]
    pair_df = spread.reset_index()
    pair_df['Date'] = pd.to_datetime(pair_df['Date'])  # Convert Date column to datetime
    pair_df.set_index('Date', inplace=True)
    pair_df['Ticker Pair'] = f'{ticker_1}-{ticker_2}'
    pair_df.columns = ['Return', 'Ticker Pair']
    spread_list.append(pair_df[['Ticker Pair', 'Return']])

return_spread_df = pd.concat(spread_list)

print(return_spread_df)

           Ticker Pair    Return
Date                            
2016-01-10   AAPL-MSFT       NaN
2016-01-17   AAPL-MSFT  0.027360
2016-01-24   AAPL-MSFT  0.018672
2016-01-31   AAPL-MSFT -0.093776
2016-02-07   AAPL-MSFT  0.060624
...                ...       ...
2018-12-02   TSLA-NVDA -0.052578
2018-12-09   TSLA-NVDA  0.118171
2018-12-16   TSLA-NVDA  0.029480
2018-12-23   TSLA-NVDA -0.010357
2018-12-30   TSLA-NVDA  0.012605

[1560 rows x 2 columns]


In [42]:
final_df = spread_df.merge(return_spread_df, how = 'inner', left_on = ['Date', 'Ticker Pair'], right_on = ['Date', 'Ticker Pair'],validate = 'one_to_one')

In [43]:
final_df.to_csv("outputs/spreads_weekly_large.csv", index=True)